In [1]:
from math import e
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

import os

DRIVER_PATH = '/Users/ngocp/.pyenv/versions/3.10.3/lib/python3.10/site-packages/selenium/webdriver'

def create_driver():
    options = Options()
    options.add_argument("--window-size=1920,1200")
    options.add_argument('--no-sandbox')

    driver = webdriver.Chrome(options=options)
    return driver

driver = create_driver()


In [2]:
import csv

class CSVWriter:
    def __init__(self, csv_file_path, fieldnames):
        self.csv_file_path = csv_file_path
        self.fieldnames = fieldnames

        # Write headers if the file doesn't exist
        file_exists = False
        try:
            with open(csv_file_path, mode='x'):
                pass
        except FileExistsError:
            file_exists = True

        if not file_exists:
            with open(csv_file_path, mode='w', newline='') as csv_file:
                writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
                writer.writeheader()

    def push_item(self, data_dict):
        with open(self.csv_file_path, mode='a', newline='') as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=self.fieldnames)
            writer.writerow(data_dict)

    def truncate_file(self):
        try:
            with open(self.csv_file_path, mode='w', newline='') as csv_file:
                csv_file.write('')
            print(f"CSV file '{self.csv_file_path}' has been truncated.")
        except Exception as e:
            print(f"Error truncating CSV file '{self.csv_file_path}': {e}")


fieldnames = ["title", "url", "content", "date", "tags"]
csv_file_path = "data.csv"

csv_writer = CSVWriter(csv_file_path, fieldnames)

In [3]:
from importlib import reload
from time import sleep
from regex import F
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import dateparser
import crawl
reload(crawl)
import pandas as pd

class Crawl:
    csv_writer = None
    def __init__(self, csv_backup = {'file_path': None}) -> None:
        self.csv_writer = CSVWriter(csv_backup['file_path'], fieldnames=["title", "url"])
        pass

class CrawlVietStock(Crawl):
    limit_page = 20
    driver = None
    links = []
    titles = []
    retry_backup = False
    source = "https://vietstock.vn/chung-khoan.htm"
    base_url = "https://vietstock.vn"

    def __init__(self, driver, limit_page = 20) -> None:
        self.driver = driver
        self.limit_page = limit_page
        self.csv_writer = CSVWriter('backup_title_url.csv', fieldnames=["title", "url"])
        pass

    def exist_record(self, url):
        query = crawl.CrawlData.select().where(crawl.CrawlData.url == url)
        return len(query) > 0

    """
    daily: True if you want to crawl only page not exist in database
    retry_backup: True if you want to retry from backup file (link, title)
    fresh_start: True if you want to truncate backup file and start from beginning
    """
    def run(self, range_date = {'start_date': None, 'end_date': None},
            daily = False, retry_backup = False,
            fresh_start = True):
        page = 1
        self.retry_backup = retry_backup
        self.driver.get(self.source)
        if fresh_start:
            self.csv_writer.truncate_file()

        if retry_backup:
            url_backup = pd.read_csv('backup_title_url.csv')
            self.links = url_backup['url'].tolist()
            self.titles = url_backup['title'].tolist()
            print('self.links ', self.links[:20], self.titles[:20])
            self.get_page_detail()
            return

        if range_date['start_date'] is not None and daily is not True:
            dateRangeElem = driver.find_element(By.CSS_SELECTOR, 'input[name="daterange"]')
            dateRangeElem.clear()
            dateRangeElem.send_keys(range_date['start_date'] + ' - ' + range_date['end_date'])
            dateRangeElem.click()
            driver.execute_script("$('.AddStockCode').remove();")
            driver.execute_script("$('div.trending-fixed').remove();")
            driver.find_element(By.CSS_SELECTOR, '.range_inputs .applyBtn').click()

        while True:
            headings_a = driver.find_elements(By.CSS_SELECTOR, "#channel-container .single_post_text h4 a")
            print('headings_a:len', len(headings_a))
            if len(headings_a) == 0:
                break

            for heading_elem in headings_a:
                try:
                    print('heading_elem', heading_elem)
                    title = heading_elem.text
                    link_detail = heading_elem.get_attribute("href")
                    print("Title:", title)
                    print("Link:", link_detail)
                    self.csv_writer.push_item({
                        'title': title,
                        'url': link_detail
                    })
                    if daily and self.exist_record(link_detail):
                        self.get_page_detail()
                        return
                    self.links.append(link_detail)
                    self.titles.append(title)
                except Exception as e:
                    print('run:e:', e)
                    continue

            # turn off popup trending
            btnTrending = driver.find_elements(By.CSS_SELECTOR, '#button-trending')
            if btnTrending:
                btnTrending[0].click()

            driver.execute_script("$('div.trending-fixed').remove();")

            pageCSSSelector = '#content-paging .next'
            pageNext = driver.find_elements(By.CSS_SELECTOR, pageCSSSelector)
            if len(pageNext) != 0:
                pageNext[0].click()
            else:
                break

            sleep(3)
            if self.limit_page != -1 and page == self.limit_page:
                break
            page += 1

    def get_page_detail(self):
        for key, link_detail in enumerate(self.links):
            try:
                print('get_page_detail', key, link_detail)
                driver.get(link_detail)
                contentElem = driver.find_element(By.CSS_SELECTOR, '.single_post_heading')
                content = contentElem.text
                dateElem = driver.find_element(By.CSS_SELECTOR, '.blog-single-head .date')

                datePublish = None
                if dateElem:
                    date = dateElem.text
                    datePublish = dateparser.parse(date)
                
                crawl.create_article({
                    'domain': 'https://vietstock.vn/chung-khoan.htm',
                    'title': self.titles[key],
                    'url': link_detail,
                    'date': datePublish,
                    'content': content,
                }, True)
            except Exception as e:
                print('get_page_detail:e:', e)
                continue


CrawlVietStock(driver, limit_page=-1).run(range_date={'start_date': '2023-05-01', 'end_date': '2023-05-31'},
    retry_backup=False, fresh_start=True, daily=True)

CSV file 'backup_title_url.csv' has been truncated.
headings_a:len 10
heading_elem <selenium.webdriver.remote.webelement.WebElement (session="ce159b197a529202f1a44086eb52e4ee", element="4E7295D024EEDFFE5E146C8A99FFCD66_element_132")>
Title: Giao dịch quỹ đầu tư: Lực bán áp đảo
Link: https://vietstock.vn/2023/08/giao-dich-quy-dau-tu-luc-ban-ap-dao-3358-1102520.htm
